In [ ]:
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize.casual import TweetTokenizer
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob
from nltk.corpus import stopwords
from collections import Counter
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
stop_words = set(stopwords.words('english'))
data = pd.read_csv('data/train.csv')

In [ ]:
def get_hashtag_column(dataframe):
    hashtags = []
    for text in dataframe.text:
        result = re.findall('#\w+', text)
        if result != []:
            result = [w[1:].lower() for w in result]
            hashtags.append(' '.join(result))
    return hashtags

In [ ]:
def lemmatize_texts(texts):
    lemmatizer = WordNetLemmatizer()
    result = []
    for t in texts:
        lemmatized_words = []
        t = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
                  'url', t)
        #t = re.sub('\!+', '', t)
        #t = re.sub('\?+', '', t)
        #t = re.sub('\d+[\:|\.]?\d*\s')
        t = re.sub('\d+', '', t)
        tokens = re.findall('''\d+,\d+|\w+'\w+|#?\w+-?\w+|\w+\*+\w+''', t)
        
        if tokens == []:
            print(t)
        
        for token in tokens:
            if token.lower() not in stop_words:
                lemmatized_words.append(lemmatizer.lemmatize(token).lower())
        result.append(' '.join(lemmatized_words).replace('#', ''))
    return result

In [ ]:
def tokenizer(text):
    return text.split(' ')

In [ ]:
all_lemmatized_texts = lemmatize_texts(data.text)
all_lemmatized_tokens = [w for t in all_lemmatized_texts for w in t.split(' ')]
print('Total words: ', len(all_lemmatized_tokens))
print('Unique_words: ', len(set(all_lemmatized_tokens)))

In [ ]:
# Most common words in dataset
freq = nltk.probability.FreqDist(all_lemmatized_tokens)
freq.most_common(20)

In [ ]:
# Most common words in real tweets
real_tweets = data[data.target == 1].text
real_tweets = lemmatize_texts(real_tweets)
freq_real = nltk.probability.FreqDist([w for t in real_tweets for w in t.split(' ')])
freq_real.most_common(10)

In [ ]:
# Most common words in fake tweets
fake_tweets = data[data.target == 0].text
fake_tweets = lemmatize_texts(fake_tweets)
freq_fake = nltk.probability.FreqDist([w for t in fake_tweets for w in t.split(' ')])
freq_fake.most_common(10)

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(all_lemmatized_texts, 
                                                    data.target, test_size=0.2, random_state=42)

In [ ]:
# Vectorize texts
vectorizer = CountVectorizer(ngram_range=(1,2), tokenizer=tokenizer)
train = vectorizer.fit_transform(X_train)
test = vectorizer.transform(X_test)

In [ ]:
# Train SVM (accuracy: 0.78)
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, tol=1e-5, max_iter=5000)
clf.fit(train, y_train)
clf.score(test, y_test)